## Setup

# DecVAE Tutorial: SimVowels Dataset

Complete workflow example for the SimVowels dataset.

In [1]:
# Import necessary libraries
import os
import json
from pathlib import Path

# Set the working directory to the DecVAE root
# Adjust this path to your local DecVAE directory
DECVAE_ROOT = Path(os.getcwd()).parent if 'examples' in os.getcwd() else Path(os.getcwd())
os.chdir(DECVAE_ROOT)
print(f"Working directory: {os.getcwd()}")

Working directory: /home/giannis/Documents/DecVAE


## 1. Generate SimVowels Dataset

Generate from scratch:

In [3]:
!python scripts/simulations/simulated_vowels.py

Succesfully saved training set part  1
Succesfully saved dev set
Succesfully saved test set


Or download from [Google Drive](https://drive.google.com/drive/folders/1VE4mkC3P1GEDrorThmRgL07NdEoLtyf9?usp=sharing).

## 2. Input Visualization

We generate input visualizations for the raw audio signal (X), and the components after applying a decomposition. We visualize individual components (OC1, OC2, ..., OCn) and aggregated representations, e.g. concatenation of all components and initial X [X,OC1,OC2,...,OCn]. We color the representations using frequency correspondence of the inputs or generative factors (vowel, speaker).

For the development set (500 utterances) of SimVowels, we will visualize the inputs to all models.

Frame-level:

In [3]:
# Visualize frame-level inputs
!accelerate launch scripts/visualize/low_dim_vis_input.py \
    --config_file config_files/input_visualizations/config_visualizing_input_frames_vowels.json

/home/giannis/anaconda3/envs/DecSSL/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Added /home/giannis/Documents/DecVAE to Python path
loading configuration file preprocessor_config.json from cache at /home/giannis/.cache/huggingface/hub/models--patrickvonplaten--wav2vec2-base-v2/snapshots/9371f1849947b4613f451680a8e96d907617ce86/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

Map (num_proc=8):   0%|                         | 0/4000 [00:00<?, ? examples/s]/home/giannis/Documents/DecVAE/models/decomposition_masking.py:1151: RuntimeWarning: invalid value encountered in scalar divide
  correlogram[o,r] = np.cov(OCs[o],OCs[r])[0,1] / (np.std(OCs[o])*np.std(OCs[r]))
Map (num_

Sequence-level:

In [ ]:
# Visualize sequence-level inputs
!accelerate launch scripts/visualize/low_dim_vis_input.py \
    --config_file config_files/input_visualizations/config_visualizing_input_sequences_vowels.json

## 3. Pre-training DecVAE

Single-GPU:

In [ ]:
# Pre-train DecVAE on single GPU
!accelerate launch scripts/pre-training/base_models_ssl_pretraining.py \
    --config_file config_files/DecVAEs/sim_vowels/pre-training/config_pretraining_sim_vowels_NoC3.json

Multi-GPU (specify GPU IDs):

In [ ]:
# Pre-train DecVAE on multiple GPUs (e.g., GPU 0 and 1)
# Uncomment and modify as needed:
# !accelerate launch --gpu_ids 0,1 scripts/pre-training/base_models_ssl_pretraining.py \
#     --config_file config_files/DecVAEs/sim_vowels/pre-training/config_pretraining_sim_vowels_NoC3.json

View configuration:

In [ ]:
import json

with open("config_files/DecVAEs/sim_vowels/pre-training/config_pretraining_sim_vowels_NoC3.json", 'r') as f:
    config = json.load(f)

print(json.dumps(config, indent=2))

## 4. Latent Evaluation

In [ ]:
# Evaluate latent representations
!accelerate launch scripts/post-training/latents_post_analysis.py \
    --config_file config_files/DecVAEs/sim_vowels/latent_evaluations/config_latent_anal_sim_vowels.json

## 5. Latent Visualization

Frame-level:

In [ ]:
# Visualize frame-level latent representations
!accelerate launch scripts/visualize/low_dim_vis_latents.py \
    --config_file config_files/DecVAEs/sim_vowels/latent_visualizations/config_latent_frames_visualization_vowels.json

Sequence-level:

In [ ]:
# Visualize sequence-level latent representations
!accelerate launch scripts/visualize/low_dim_vis_latents.py \
    --config_file config_files/DecVAEs/sim_vowels/latent_visualizations/config_latent_sequences_visualization_vowels.json

## 6. Latent Traversals

Generate controlled dataset:

In [ ]:
# Generate small-scale dataset with controlled generative factors
!python scripts/simulations/simulated_vowels_for_latent_traversal.py

Perform traversal analysis:

In [ ]:
# Perform latent traversal analysis
!accelerate launch scripts/latent_response_analysis/latent_traversal_analysis.py \
    --config_file config_files/DecVAEs/sim_vowels/latent_traversals/config_latent_traversals_sim_vowels.json